# task description

task 1 - write script that takes POSCAR and mde workspace as input and plots dispersions along high symmetry path
* write function where you give space group and it outputs you a path (seekpath package)
* choose some offset by whole int BZ (depending on coverage, signal strength, rings, etc.); can be different for each slice; care about structure (worry about intens later)
* make a slice for each with mantid and save workspace mde
* need to scale each portion so that it is in terms of angstrom
* need to figure len of each hist so that sym points line up on the plot

* in the future, could try to automate the step where you are choosing the offset (i.e. check coverage, check for rings, high q for phonon / low for magnetism, etc.)

task 2
* also, how to export the histogram as np array and import to python

# setup

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seekpath
from seekpath.util import atoms_num_dict

In [ ]:
from matplotlib.gridspec import GridSpec
from mantid.simpleapi import CreateMDHistoWorkspace
from mantid import plots

import sys
sys.path.append('/SNS/groups/dgs/DGS_SC_scripts')
from reduce_data_to_MDE import *
from slice_utils import *
import define_data
import define_slices
import os
import imp

# read poscar for seekpath input

In [4]:
def simple_read_poscar(fname):
    """Read a POSCAR file."""
    with open(fname) as f:
        lines = [l.partition("!")[0] for l in f.readlines()]

    alat = float(lines[1])
    v1 = [float(_) * alat for _ in lines[2].split()]
    v2 = [float(_) * alat for _ in lines[3].split()]
    v3 = [float(_) * alat for _ in lines[4].split()]
    cell = [v1, v2, v3]

    species = lines[5].split()
    num_atoms = [int(_) for _ in lines[6].split()]

    next_line = lines[7]
    if next_line.strip().lower() != "direct":
        raise ValueError("This simple routine can only deal with 'direct' POSCARs")
    # Note: to support also cartesian, remember to multiply the coordinates by alat

    positions = []
    atomic_numbers = []
    cnt = 8
    for el, num in zip(species, num_atoms):
        atom_num = atoms_num_dict[el.capitalize()]
        for _ in range(num):
            atomic_numbers.append(atom_num)
            positions.append([float(_) for _ in lines[cnt].split()])
            cnt += 1

    return (cell, positions, atomic_numbers)

In [5]:
poscar = simple_read_poscar('/SNS/users/ajs192/Desktop/Research/MDsim_6.30.23/dataViz/POSCAR')
print(poscar)

([[4.65547841329003, -0.0, 0.0], [0.0, 4.65547841329003, -0.0], [0.0, -0.0, 4.65547841329003]], [[0.13476038461486, 0.13476038461486, 0.13476038461486], [0.36523961538514, 0.86523961538514, 0.63476038461486], [0.86523961538514, 0.63476038461486, 0.36523961538514], [0.63476038461486, 0.36523961538514, 0.86523961538514], [0.84874890814554, 0.84874890814554, 0.84874890814554], [0.65125109185446, 0.15125109185446, 0.34874890814554], [0.15125109185446, 0.34874890814554, 0.65125109185446], [0.34874890814554, 0.65125109185446, 0.15125109185446]], [26, 26, 26, 26, 14, 14, 14, 14])


# determining high symmetry path

FeSi unit cell:

<img src="FeSi_unitCell.png" alt="Alternative text" />

In [6]:
path = seekpath.get_path(structure=poscar)
print(path['path'])
print(path['point_coords'])
print(path.keys())

print(path['bravais_lattice'])

[('GAMMA', 'X'), ('X', 'M'), ('M', 'GAMMA'), ('GAMMA', 'R'), ('R', 'X'), ('R', 'M'), ('M', 'X_1')]
{'GAMMA': [0.0, 0.0, 0.0], 'R': [0.5, 0.5, 0.5], 'M': [0.5, 0.5, 0.0], 'X': [0.0, 0.5, 0.0], 'X_1': [0.5, 0.0, 0.0]}
dict_keys(['point_coords', 'path', 'has_inversion_symmetry', 'augmented_path', 'bravais_lattice', 'bravais_lattice_extended', 'conv_lattice', 'conv_positions', 'conv_types', 'primitive_lattice', 'primitive_positions', 'primitive_types', 'reciprocal_primitive_lattice', 'inverse_primitive_transformation_matrix', 'primitive_transformation_matrix', 'volume_original_wrt_conv', 'volume_original_wrt_prim', 'spacegroup_number', 'spacegroup_international', 'rotation_matrix'])
cP


Seekpath for FeSi matches the path given by aflow: https://aflow.org/material/?id=aflow:36503a08caf0a17f

Can look at path and BZ in seekpath online tool + visualizer: https://www.materialscloud.org/work/tools/seekpath

<img src="FeSi_highSymPath1_sm.png" alt="Alternative text" />

<img src="FeSi_highSymPath_sm.png" alt="Alternative text" />

# choosing dims and bins for slices

Need to be able to choose bin dimensions based on the given symmetry endpoints of each path segment. Work through some example by hand to try and understand the algorithm

* For (GAMMA,X)=[(0,0,0),(0,0.5,0)]   !!!MAYBE WRONG!!!

'QDimension0':'1,1,0',

'QDimension1':'-1,1,0',

'QDimension2':'0,0,1',

<br>
'Dimension0Binning':'-0.15,0.15',

'Dimension1Binning':'-0.15,0.15',

'Dimension2Binning':'0,0.025,0.5'

<br>

* For (X,M)=[(0,0.5,0), (0.5,0.5,0)]        !!!ON PAPER!!!

'QDimension0':'1,1,0',

'QDimension1':'-1,1,0',

'QDimension2':'0,0,1',

<br>
'Dimension0Binning':'3.85,4.15',

'Dimension1Binning':'-0.15,0.15',

'Dimension2Binning':'-7.3625,0.025,8.4875'


In [7]:
print(path['path'][0])
print(path['point_coords']['GAMMA'])
print(path['point_coords']['X'])

print('test')
point_cord = path['point_coords']
path_seg = path['path'][0]
path_brav = path['bravais_lattice']
print(path_seg[0])
print(point_cord[path_seg[0]])
print(path_brav)

('GAMMA', 'X')
[0.0, 0.0, 0.0]
[0.0, 0.5, 0.0]
test
GAMMA
[0.0, 0.0, 0.0]
cP


In [8]:
diff = np.array(point_cord[path_seg[1]]) - np.array(point_cord[path_seg[0]])
print(diff)
print(np.nonzero(diff)[0])
scaler = 1/diff[np.nonzero(diff)[0]]
qdim0_range = np.array([-1,0.025,1])
print(qdim0_range*scaler)

[0.  0.5 0. ]
[1]
[-2.    0.05  2.  ]


In [9]:
diff3d = np.array([0.3,0,0.75])
print(diff3d)
print(np.nonzero(diff3d)[0])
minDiffNonZero = np.min(diff3d[np.nonzero(diff3d)[0]])
scaler = 1/minDiffNonZero
def_range = np.array([-1,0.025,1])
qdim0_range = def_range*scaler
print(qdim0_range)

[0.3  0.   0.75]
[0 2]
[-3.33333333  0.08333333  3.33333333]


In [10]:
def find_qdim0_range(qdim0):
    # calculating default QDimension0 range and bin size
    # bounds chosen to cover first BZ and step is from most mantid examples
    def_range = np.array([-0.5,0.025,0.5])
    minDiffNonZero = np.min(qdim0[np.nonzero(qdim0)[0]])
    # scaler makes sure resolution stays normal with weird qdim0
    scaler = 1/minDiffNonZero
    qdim0_range = def_range*scaler
    return qdim0_range

In [11]:
h = np.array([1,0,0])
k = np.array([0,1,0])
l = np.array([0,0,1])

#qdim0 = np.array(point_cord[path_seg[1]]) - np.array(point_cord[path_seg[0]])
qdim0 = np.array([0.3,0,0.75])
unit_qdim0 = qdim0 / np.linalg.norm(qdim0)
print(unit_qdim0)
print("h parallel score: {}".format(np.dot(unit_qdim0,h)))
print("k parallel score: {}".format(np.dot(unit_qdim0,k)))
print("l parallel score: {}".format(np.dot(unit_qdim0,l)))


[0.37139068 0.         0.92847669]
h parallel score: 0.3713906763541037
k parallel score: 0.0
l parallel score: 0.9284766908852594


Helpful links:
* Angle btw two vecs: https://www.emathhelp.net/en/calculators/linear-algebra/angle-between-two-vectors-calculator/?u=0.25%2C0.75%2C0.5&v=0.75%2C-0.25%2C0

* Cross product calc: https://www.symbolab.com/solver/vector-cross-product-calculator/%5Cbegin%7Bpmatrix%7D0.25%260.75%260.5%5Cend%7Bpmatrix%7D%5Ctimes%5Cbegin%7Bpmatrix%7D0.75%26-0.25%260%5Cend%7Bpmatrix%7D?or=input

* Find plane eq given 3 points: https://keisan.casio.com/exec/system/1223596129

* Geogebra 3D viz: https://www.geogebra.org/3d?lang=en 

In [12]:
def calc_para_scores(qdim0):
    hkl = np.array([[1,0,0],[0,1,0],[0,0,1]])
    unit_qdim0 = qdim0 / np.linalg.norm(qdim0)
    # calculate degree of parallelism btw qdim0 and hkl vecs
    para_scores = np.zeros((3,1))
    for i in range(3):
        para_scores[i] = np.dot(unit_qdim0,hkl[i])
    return para_scores



def find_qdim_1and2(qdim0, path_bravLatt, choose_cubic_QDims=False):
    hkl = np.array([[1,0,0],[0,1,0],[0,0,1]])
    
    # check if bravais lattice is cubic and if 
    # non-default, cubic qdim1,2 selection is wanted
    if (path_bravLatt[0]=='c') and (choose_cubic_QDims):
        if np.count_nonzero(qdim0) == 1:
            # if diff=dim0 is 1D, then dim1 and dim2 will also just be 1D h,k,or l
            qdim0_dir = np.nonzero(qdim0)[0]
            hkl_avail = np.delete(hkl, qdim0_dir, axis=0)
            qdim1 = hkl_avail[0]
            qdim2 = hkl_avail[1]
        elif (np.count_nonzero(qdim0) == 2) or (np.count_nonzero(qdim0) == 3):
            nzeroInds = np.nonzero(qdim0)[0]
            qdim1 = qdim0.copy()
            # swap h and k and make one negative to give perpen direction within the horizontal plane
            # If qdim0 is 3D, need to set l to 0 so that it is perpen and in the horizontal plane 
            qdim1[nzeroInds[0]], qdim1[nzeroInds[1]] = -qdim0[nzeroInds[1]], qdim0[nzeroInds[0]]
            if np.count_nonzero(qdim0) == 3:
                qdim1[2] = 0 
            # qdim2 is simply the direction perpen to qdim0 and qdim1
            qdim2 = np.cross(qdim0, qdim1)
            # normalize the two qdims so that mag of each is 1 A-1 and can easily do bins +/- 0.15
            qdim1 = qdim1 / np.linalg.norm(qdim1)
            qdim2 = qdim2 / np.linalg.norm(qdim2)
        else:
            raise ValueError("The selected symmetry path contains a segment which is greater than 3 dimensions") 
    else:
        # if the lattice isn't cubic and/or the default qdim selection is wanted, do it based on which 2 hkl are most perp (i.e. least para) to dim0
        para_scores = calc_para_scores(qdim0)
        # want two directions most perpen to qdim0 to be qdim1 and qdim2
        qdim1_ind = np.argmin(para_scores)
        qdim1 = hkl[qdim1_ind]
        # set qdim1 as highest so it won't be chosen for qdim2
        para_scores[qdim1_ind] = 2
        qdim2_ind = np.argmin(para_scores)
        qdim2 = hkl[qdim2_ind]
    return qdim1, qdim2

tests

In [13]:
qdim0 = np.array([0.6,0,0.75])
qdim1,qdim2 = find_qdim_1and2(qdim0, path_brav, choose_cubic_QDims=True)
print(qdim1,qdim2)

qdim0 = np.array([0.6,0,0.75])
qdim1,qdim2 = find_qdim_1and2(qdim0, path_brav, choose_cubic_QDims=False)
print(qdim1,qdim2)

print("\ntest2:")
qdim0 = np.array([0.5,0.5,0])
qdim1,qdim2 = find_qdim_1and2(qdim0, path_brav, choose_cubic_QDims=True)
print(qdim1,qdim2)

[-0.78086881  0.          0.62469505] [ 0. -1.  0.]
[0 1 0] [1 0 0]

test2:
[-0.70710678  0.70710678  0.        ] [ 0. -0.  1.]


dev

In [14]:
#qdim0tst = np.array([0.25,0.75,0.5])
qdim0tst = np.array([0.5,0.5,0.5])
#qdim0tst = np.array([1,0,0])
hkl = np.array([[1,0,0],[0,1,0],[0,0,1]])
zeroInds = np.where(qdim0tst == 0)[0]
nzeroInds = np.nonzero(qdim0tst)[0]
print(zeroInds)
print(nzeroInds)
qdim1tst = qdim0tst.copy()
qdim1tst[nzeroInds[0]], qdim1tst[nzeroInds[1]] = -qdim0tst[nzeroInds[1]], qdim0tst[nzeroInds[0]]
if np.count_nonzero(qdim0tst) == 3:
    qdim1tst[2] = 0 
qdim2tst = np.cross(qdim0tst, qdim1tst)
qdim1tst = qdim1tst / np.linalg.norm(qdim1tst)
qdim2tst = qdim2tst / np.linalg.norm(qdim2tst)
print(qdim0tst)
print(qdim1tst)
print(qdim2tst)

[]
[0 1 2]
[0.5 0.5 0.5]
[-0.70710678  0.70710678  0.        ]
[-0.40824829 -0.40824829  0.81649658]


driver function for dim and bin selection

In [15]:
def choose_dims_and_bins(path_segment, point_coords, path_bravLatt, choose_cubic_QDims=False):
    
    pt1 = np.array(point_coords[path_segment[0]])
    pt2 = np.array(point_coords[path_segment[1]])
    diff = pt2 - pt1

    # calculating default QDimension0 axis direction, range, and bin size
    qdim0 = diff
    qdim0_range = find_qdim0_range(qdim0)

    # determine the directions of qdim1 and qdim2
    qdim1,qdim2 = find_qdim_1and2(qdim0,path_bravLatt,choose_cubic_QDims)
    qdim1_range = [-0.15,0.15]
    qdim2_range = [-0.15,0.15] 

    # return list contains various types (mainly np arrays) that are converted to properly formatted strings
    # as a part of the slice description generation process
    q_dims_and_bins = [qdim0, qdim1, qdim2, qdim0_range, qdim1_range, qdim2_range]
    return q_dims_and_bins


driver test

In [26]:
path_seg = path['path'][3]
point_cord = path['point_coords']
path_brav = path['bravais_lattice']
print(path_seg)
print(point_cord)
print(path_brav)

q_dims_and_bins = choose_dims_and_bins(path_seg,point_cord,path_brav,choose_cubic_QDims=True)
print(q_dims_and_bins)

('GAMMA', 'R')
{'GAMMA': [0.0, 0.0, 0.0], 'R': [0.5, 0.5, 0.5], 'M': [0.5, 0.5, 0.0], 'X': [0.0, 0.5, 0.0], 'X_1': [0.5, 0.0, 0.0]}
cP
[array([0.5, 0.5, 0.5]), array([-0.70710678,  0.70710678,  0.        ]), array([-0.40824829, -0.40824829,  0.81649658]), array([-1.  ,  0.05,  1.  ]), [-0.15, 0.15], [-0.15, 0.15]]


# making slice descriptions

In [46]:
test_slice={'QDimension0':'1,1,0',
            'QDimension1':'-1,1,0',
            'QDimension2':'0,0,1',
            'Dimension0Name':'QDimension0',
            'Dimension0Binning':'3.85,4.15',
            'Dimension1Name':'QDimension1',
            'Dimension1Binning':'-0.15,0.15',
            'Dimension2Name':'QDimension2',
            'Dimension2Binning':'-7.3625,0.025,8.4875',
            'Dimension3Name':'DeltaE',
            'Dimension3Binning':'-35.5,1,66.5',
            'SymmetryOperations':'x,y,z'}

In [47]:
def convToDescString(input_item):
    # works for np arrays, lists, and tuples
    string = ''
    for item in input_item:
        string = string + str(item) +','
    string = string[:-1]
    return string

In [48]:
print(test_slice['QDimension0'])
qdim2= 1,1,0
tst_str = convToDescString(qdim2)
print(tst_str)
print(test_slice['QDimension0'] == tst_str)

dim3= [1,1,0]
tst_str2 = convToDescString(dim3)
print(tst_str2)
print(test_slice['QDimension0'] == tst_str2)

dim4= np.array([1,1,0])
tst_str4 = convToDescString(dim4)
print(tst_str4)
print(test_slice['QDimension0'] == tst_str4)

1,1,0
1,1,0
True
1,1,0
True
1,1,0
True


replacing dim in slice desc dict with the converted one

In [49]:
print(test_slice['QDimension0'])
dim3= [1,2,0]
tst_str2 = convToDescString(dim3)
print(tst_str2)
test_slice['QDimension0'] = tst_str2
print(test_slice['QDimension0'])

1,1,0
1,2,0
1,2,0


function to make slice description

In [56]:
def make_slice_desc(q_dims_and_bins):
    # baseline slice description to be edited
    slice_desc={'QDimension0':'1,0,0',
                'QDimension1':'0,1,0',
                'QDimension2':'0,0,1',
                'Dimension0Name':'QDimension0',
                'Dimension0Binning':'0,0.025,0.5',
                'Dimension1Name':'QDimension1',
                'Dimension1Binning':'-0.15,0.15',
                'Dimension2Name':'QDimension2',
                'Dimension2Binning':'-0.15,0.15',
                'Dimension3Name':'DeltaE',
                'Dimension3Binning':'-35.5,1,66.5',
                'SymmetryOperations':'x,y,z'}
    
    # replacing defaults with the desired q dims and bins
    slice_desc['QDimension0'] = convToDescString(q_dims_and_bins[0])
    slice_desc['QDimension1'] = convToDescString(q_dims_and_bins[1])
    slice_desc['QDimension2'] = convToDescString(q_dims_and_bins[2])
    slice_desc['Dimension0Binning'] = convToDescString(q_dims_and_bins[3])
    slice_desc['Dimension1Binning'] = convToDescString(q_dims_and_bins[4])
    slice_desc['Dimension2Binning'] = convToDescString(q_dims_and_bins[5])

    return slice_desc



test

In [57]:
exSlice = make_slice_desc(q_dims_and_bins)
print(exSlice)

{'QDimension0': '0.0,0.5,0.0', 'QDimension1': '1,0,0', 'QDimension2': '0,0,1', 'Dimension0Name': 'QDimension0', 'Dimension0Binning': '-1.0,0.05,1.0', 'Dimension1Name': 'QDimension1', 'Dimension1Binning': '-0.15,0.15', 'Dimension2Name': 'QDimension2', 'Dimension2Binning': '-0.15,0.15', 'Dimension3Name': 'DeltaE', 'Dimension3Binning': '-35.5,1,66.5', 'SymmetryOperations': 'x,y,z'}


# take slice w/ Mantid python API

# scale q and plot many slices together

In [ ]:
# https://docs.mantidproject.org/nightly/plotting/index.html#plotting 
# Manually plot the path for the FeSi data
m_gamma = np.sqrt(2)
gamma_r = np.sqrt(3)
r_x = np.sqrt(2)

fig=plt.figure(figsize=(12,5))
gs = GridSpec(1, 5,width_ratios=[1,1,m_gamma,gamma_r,r_x],wspace=0)

ax1 = plt.subplot(gs[0],projection='mantid')
ax2 = plt.subplot(gs[1],sharey=ax1,projection='mantid')
ax3 = plt.subplot(gs[2],sharey=ax1,projection='mantid')
ax4 = plt.subplot(gs[3],sharey=ax1,projection='mantid')
ax5 = plt.subplot(gs[4],sharey=ax1,projection='mantid')


colormesh_pars={}
colormesh_pars['norm']=LogNorm(vmin=1e-6,vmax=1)

mtd.importAll()
ax1.pcolormesh(FeSi_300K_Ei70meV_GAMMA2X, **colormesh_pars)
ax2.pcolormesh(FeSi_300K_Ei70meV_X2M, **colormesh_pars)
ax3.pcolormesh(FeSi_300K_Ei70meV_M2GAMMA, **colormesh_pars)
ax4.pcolormesh(FeSi_300K_Ei70meV_GAMMA2R, **colormesh_pars)
ax5.pcolormesh(FeSi_300K_Ei70meV_R2X, **colormesh_pars)

#Adjust plotting parameters

ax1.set_ylabel('E (meV)')
ax1.set_xlabel('')
#ax1.set_xlim(0,1./3)
ax1.set_ylim(-10.,30.)
#ax1.set_title(r'$[\epsilon,\epsilon,0], 0 \leq \epsilon \leq 1/3$')
ax1.set_xticks([0,0.5])
ax1.set_xticklabels(['$\Gamma$','$X$'])
#ax1.spines['right'].set_visible(False)
ax1.tick_params(direction='in')

ax2.get_yaxis().set_visible(False)
#ax2.set_xlim(1./3,1./2)
ax2.set_xlabel('')
#ax2.set_title(r'$[1/3+\epsilon,1/3-2\epsilon,0], 0 \leq \epsilon \leq 1/6$')
ax2.set_xticks([0.5])
ax2.set_xticklabels(['$M$'])
#ax2.spines['left'].set_visible(False)
ax2.tick_params(direction='in')

ax3.get_yaxis().set_visible(False)
ax3.set_xlim(0.5,0)
ax3.set_xlabel('')
#ax2.set_title(r'$[1/3+\epsilon,1/3-2\epsilon,0], 0 \leq \epsilon \leq 1/6$')
ax3.set_xticks([0])
ax3.set_xticklabels(['$\Gamma$'])
#ax2.spines['left'].set_visible(False)
ax3.tick_params(direction='in')

ax4.get_yaxis().set_visible(False)
#ax2.set_xlim(1./3,1./2)
ax4.set_xlabel('')
#ax2.set_title(r'$[1/3+\epsilon,1/3-2\epsilon,0], 0 \leq \epsilon \leq 1/6$')
ax4.set_xticks([0.5])
ax4.set_xticklabels(['$R$'])
#ax2.spines['left'].set_visible(False)
ax4.tick_params(direction='in')

ax5.get_yaxis().set_visible(False)
#ax2.set_xlim(1./3,1./2)
ax5.set_xlabel('')
#ax2.set_title(r'$[1/3+\epsilon,1/3-2\epsilon,0], 0 \leq \epsilon \leq 1/6$')
ax5.set_xticks([0.5])
ax5.set_xticklabels(['$X$'])
#ax2.spines['left'].set_visible(False)
ax5.tick_params(direction='in')

mid = (fig.subplotpars.right + fig.subplotpars.left)/2
plt.suptitle('FeSi S(Q,E) along high symmetry path in 1st BZ',x=mid)

fig.show()

og code 

In [ ]:
# Generate nice (fake) dispersion data
# Gamma to K
q = np.arange(0,0.333,0.01)
e = np.arange(0,60)
x,y = np.meshgrid(q,e)
omega_hh = 20. * np.sin(np.pi*x*1.5)
I_hh = np.exp(-x*5.)
signal = I_hh * np.exp(-(y-omega_hh)**2)
signal[y>25+100*x**2]=np.nan
ws1=CreateMDHistoWorkspace(Dimensionality=2,
                           Extents='0,0.3333,0,60',
                           SignalInput=signal,
                           ErrorInput=np.sqrt(signal),
                           NumberOfBins='{0},{1}'.format(len(q),len(e)),
                           Names='qdim1,qdim2',
                           Units='MomentumTransfer,EnergyTransfer')
# K to M
q = np.arange (0.333,0.5, 0.01)
x,y = np.meshgrid(q,e)
omega_hm2h=20. * np.cos(np.pi*(x-0.333))
signal = np.exp(-(y-omega_hm2h)**2)
signal[y>35]=np.nan
ws2=CreateMDHistoWorkspace(Dimensionality=2,
                           Extents='0.3333,0.5,0,60',
                           SignalInput=signal,
                           ErrorInput=np.sqrt(signal),
                           NumberOfBins='{0},{1}'.format(len(q),len(e)),
                           Names='qdim1,qdim2',
                           Units='MomentumTransfer,EnergyTransfer')


d=6.7
a=2.454
#Gamma is (0,0,0)
#A is (0,0,1/2)
#K is (1/3,1/3,0)
#M is (1/2,0,0)
gamma_a=np.pi/d
gamma_m=2.*np.pi/np.sqrt(3.)/a
m_k=2.*np.pi/3/a
gamma_k=4.*np.pi/3/a

fig=plt.figure(figsize=(12,5))
gs = GridSpec(1, 4,
              width_ratios=[gamma_k,m_k,gamma_m,gamma_a],
              wspace=0)

ax1 = plt.subplot(gs[0],projection='mantid')
ax2 = plt.subplot(gs[1],sharey=ax1,projection='mantid')
ax3 = plt.subplot(gs[2],sharey=ax1)
ax4 = plt.subplot(gs[3],sharey=ax1)

ax1.pcolormesh(ws1)
ax2.pcolormesh(ws2)
ax3.plot([0,0.5],[0,17.])
ax4.plot([0,0.5],[0,10])


#Adjust plotting parameters

ax1.set_ylabel('E (meV)')
ax1.set_xlabel('')
ax1.set_xlim(0,1./3)
ax1.set_ylim(0.,40.)
ax1.set_title(r'$[\epsilon,\epsilon,0], 0 \leq \epsilon \leq 1/3$')
ax1.set_xticks([0,1./3])
ax1.set_xticklabels(['$\Gamma$','$K$'])
#ax1.spines['right'].set_visible(False)
ax1.tick_params(direction='in')

ax2.get_yaxis().set_visible(False)
ax2.set_xlim(1./3,1./2)
ax2.set_xlabel('')
ax2.set_title(r'$[1/3+\epsilon,1/3-2\epsilon,0], 0 \leq \epsilon \leq 1/6$')
ax2.set_xticks([1./2])
ax2.set_xticklabels(['$M$'])
#ax2.spines['left'].set_visible(False)
ax2.tick_params(direction='in')

#invert axis
ax3.set_xlim(1./2,0)
ax3.get_yaxis().set_visible(False)
ax3.set_title(r'$[\epsilon,0,0], 1/2 \geq \epsilon \geq 0$')
ax3.set_xticks([0])
ax3.set_xticklabels(['$\Gamma$'])
ax3.tick_params(direction='in')

ax4.set_xlim(0,1./2)
ax4.get_yaxis().set_visible(False)
ax4.set_title(r'$[0,0,\epsilon], 0 \leq \epsilon \leq 1/2$')
ax4.set_xticks([1./2])
ax4.set_xticklabels(['$A$'])
ax4.tick_params(direction='in')
#fig.show()

func inspired by: https://github.com/giovannipizzi/seekpath/blob/main/tests/test_paths_hpkot.py